<a href="https://colab.research.google.com/github/monimoreno2905/SegundoParcial/blob/main/RedDependencia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Se plantea el modelo con la función de costos de la medida de dependencia COCO

1. LIBRERIAS GENERALES

In [ ]:
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# en este contexto numpy se utiliza para hacer algunas operaciones matemáticas y definir arreglos
import numpy as np
#En este ejercicio básicamente se utiliza para manipular directorios
import os

from tensorflow.keras import layers

# Con esto se garantiza que produzcan los mismos resultados cada vez que se ejecuta el código
np.random.seed(42)
tf.random.set_seed(42)

# Librerias para la generación de gráficas
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
from tensorflow.keras import layers, Model

2. CARGA DE DATOS

In [ ]:
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full.astype(np.float32) / 255 #normalización de los datos
X_test = X_test.astype(np.float32) / 255
X_train, X_valid = X_train_full[:-5000], X_train_full[-5000:] #Se toman solo algunos datos, no todos
y_train, y_valid = y_train_full[:-5000], y_train_full[-5000:]

3. FUNCIONES PARA CALCULAR F Y G de acuerdo a las ecuaciones del Kernel reproductor.

Para no tener problemas en las dimensiones para realizar operaciones más adelante en la función de costos, tenemos;

$f=X^{T}Hα  \in R^{784x1}$

$g=Y^{T}Hβ  \in R^{1x1}$

$C{xy}=X^{T}HY ∈ R^{784x1}$

y la función de costos evaluada

$L(f, g, \lambda, \gamma) = -f^{T} C_{XY} g + \frac{\lambda}{2} \|f\|^2_F - 1 + \frac{\gamma}{2} \|g\|^2_G - 1
$

In [ ]:
"""Función para calcular f
X son las imágenes
pesos_alpha son los pesos de la capa alpha
"""
def calculate_f(X,H,pesos_alpha):
  x_flat = tf.cast(tf.reshape(X, [tf.shape(X)[0], -1]), tf.float32)
  f = tf.linalg.matmul(tf.linalg.matmul(tf.transpose(x_flat),H), pesos_alpha)
  return f

In [ ]:
"""Función para calcular g
Y son las etiquetas de las imágenes
pesos_betha son los pesos de la capa betha
"""
def calculate_g(Y,H,pesos_Betha):
  Y = tf.cast(Y, tf.float32)
  g = tf.linalg.matmul(tf.linalg.matmul(tf.transpose(Y),H), pesos_Betha)
  return g

In [ ]:
"""Función para calcular Cxy
X son las imágenes
Y las etiquetas
"""
def calculate_Cxy(X,H,Y):
  x_flat = tf.cast(tf.reshape(X, [tf.shape(X)[0], -1]), tf.float32)
  Cxy = tf.linalg.matmul(tf.linalg.matmul(tf.transpose(x_flat),H), Y)
  return Cxy

In [ ]:
"""Función para calcular kernels"""
import tensorflow_probability as tfp

#Se importa un kernel gaussiano con el valor óptimo de sigma
sigma = 11.44  # Valor de sigma óptimo
gaussian_kernel = tfp.math.psd_kernels.ExponentiatedQuadratic(length_scale=sigma) #Kernel gaussiano ecuación 2

#Kernel aplicado a los datos
def kernel_matrix(x, y):
    # Aplanar las imágenes
    x_flat = tf.cast(tf.reshape(x, [tf.shape(x)[0], -1]), tf.float32)  # Aplana x a [x_size, m*n]
    y_flat = tf.cast(tf.reshape(y, [tf.shape(y)[0], -1]), tf.float32)  # Aplana y a [y_size, m*n]

    # Calcular la matriz del kernel
    return gaussian_kernel.matrix(x_flat, y_flat)  # Cambiado a .matrix

In [ ]:
"""Función para definir el loss"""
def loss_custom(f,g,Cxy,K,L,n):
  lambda_param = tf.reduce_sum(tf.sqrt(tf.matmul(K,L)))/n
  gamma_param = lambda_param
  J = -tf.matmul(tf.matmul(tf.transpose(f), Cxy), g) + \
    (lambda_param / 2) * tf.matmul(tf.transpose(f), f) + \
    (gamma_param / 2) * tf.square(g)

  return J

4. EVALUACIÓN EN PUNTOS INDUCIDOS

In [ ]:
#Se escogen los datos
X=X_train[::50]
Y=y_train[::50]
Y=Y.reshape(-1,1)
#Se calculan los kernels y otros parametros
n = tf.shape(X)[0]
n = tf.cast(n, tf.float32)
H = tf.eye(n, dtype=tf.float32) - (1/n) * tf.ones((n, n), dtype=tf.float32)
K = kernel_matrix(X, X) #Kernel de imágenes
L = kernel_matrix(Y, Y) #Kernel de etiquetas
#Se calculan los pesos de acuerdo a las ecuaciones de los anotes
n = tf.cast(n, tf.int32)
pesos_alpha = tf.random.normal(shape=(n, 1), mean=0.0, stddev=1.0, dtype=tf.float32)
pesos_betha=tf.matmul(tf.divide(tf.sqrt(tf.matmul(K,L)),L),pesos_alpha)
#Se aplican las funciones
f=calculate_f(X,H,pesos_alpha)
g=calculate_g(Y,H,pesos_betha)
Cxy=calculate_Cxy(X,H,Y)
#Se calcula la función de costos
n = tf.cast(n, tf.float32)
J=loss_custom(f,g,Cxy,K,L,n)
print(J)

tf.Tensor([[5.8963794e+14]], shape=(1, 1), dtype=float32)


In [ ]:
#Ahora se despeja de g calculada anteriormente el valor de Y
#A diferencia de g, todos los demás parametros vuelven a calcularse
X=X_test[::50]
Y=y_test[::50]
Y=Y.reshape(-1,1)
n = tf.shape(X)[0]
n = tf.cast(n, tf.float32)
H = tf.eye(n, dtype=tf.float32) - (1/n) * tf.ones((n, n), dtype=tf.float32)
K = kernel_matrix(X, X)
L = kernel_matrix(Y, Y)

n = tf.cast(n, tf.int32)
pesos_alpha = tf.random.normal(shape=(n, 1), mean=0.0, stddev=1.0, dtype=tf.float32)
pesos_betha=tf.matmul(tf.divide(tf.sqrt(tf.matmul(K,L)),L),pesos_alpha)
#Se obtienen las nuevas etiquetas
YNew=g/tf.matmul(H,pesos_betha)